In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from visualization_utilities import *
from metadata import *
from utilities2015 import *

Setting environment for Precision WorkStation


No vtk
No vtkNot using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


In [2]:
from pandas import *

# New version

In [12]:
contours_df = DataManager.load_annotation_v4(stack='MD642', by_human=True, timestamp='latest', suffix='contours')

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD642" "/home/yuncong/CSHL_labelings_v3/MD642" --exclude "*" --include "*contours*"
latest timestamp:  01052018123332


In [5]:
contours_df = contours_df[contours_df['filename'] != 'Placeholder']

In [58]:
contours_df['side'][contours_df['name'] == '12N'] = 'S'

In [59]:
contours_df[(contours_df['type'] == 'confirmed') & (contours_df['name'] == '12N')]

,class,creator,downsample,edits,filename,id,label_position,name,orientation,parent_structure,section,side,side_manually_assigned,time_created,type,vertices
117032276953674,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'121...",ChatCryoJane201710_slide56-S1,117032276953674,"[34120.1866687, 15879.0191139]",12N,sagittal,[],164,S,True,12122017191923,confirmed,"[[34184.4795659, 16455.8560184], [33775.840642..."
152308078665262,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'121...",ChatCryoJane201710_slide54-S2,152308078665262,"[34226.5621568, 15442.448111]",12N,sagittal,[],160,S,True,12122017191847,confirmed,"[[33839.4697218, 15748.0123964], [33541.509009..."
232696124933381,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'121...",ChatCryoJane201710_slide56-S3,232696124933381,"[34399.1878739, 13739.8026016]",12N,sagittal,[],168,S,True,12122017192022,confirmed,"[[34126.6628251, 14190.5074574], [33757.543818..."
237829324806731,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'121...",ChatCryoJane201710_slide58-S1,237829324806731,"[34771.6481868, 12579.1406533]",12N,sagittal,[],170,S,True,12122017192133,confirmed,"[[34532.4568632, 13222.5844545], [34271.814438..."


In [6]:
sagittal_contours_df_fp = DataManager.get_annotation_filepath(stack='MD642', by_human=True, suffix='contours', 
                                                              timestamp='now')
save_hdf_v2(contours_df, sagittal_contours_df_fp)
upload_to_s3(sagittal_contours_df_fp)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['class', 'creator', 'downsample', 'edits', 'filename', 'id', 'label_position', 'name', 'orientation', 'parent_structure', 'section', 'side', 'side_manually_assigned', 'time_created', 'type', 'vertices']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


aws s3 cp "/home/yuncong/CSHL_labelings_v3/MD642/MD642_annotation_contours_01052018114915.hdf" "s3://mousebrainatlas-data/CSHL_labelings_v3/MD642/MD642_annotation_contours_01052018114915.hdf"


# Old version

In [1]:
stack = 'MD635'

In [6]:
fp = '/home/yuncong/CSHL_labelings_v3/%(stack)s/%(stack)s_annotation_v3.h5' % {'stack': stack}
download_from_s3(fp)
contours = read_hdf(fp)

rm -rf "/home/yuncong/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5" && mkdir -p "/home/yuncong/CSHL_labelings_v3/MD635"
aws s3 cp "s3://mousebrainatlas-data/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5" "/home/yuncong/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5"


In [12]:
contours[(contours['name'] == '7N') & (contours['section'] == 158)]

,creator,downsample,edits,filename,flags,id,label_position,name,orientation,parent_structure,section,side,side_manually_assigned,time_created,vertices
102627349379154,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'122...",MD635-F53-2016.05.19-05.14.50_MD635_2_0158,1,102627349379154,"[25175.1667763, 12351.1854336]",7N,sagittal,[],158,L,False,12262016143425,"[[24780.2371365, 12783.8648579], [24620.295638..."
106833387670490,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'122...",MD635-F53-2016.05.19-05.14.50_MD635_2_0158,1,106833387670490,"[25175.1667763, 12351.1854336]",7N,sagittal,[],158,L,False,12262016141652,"[[24780.2371365, 12783.8648579], [24620.295638..."


In [7]:
from annotation_utilities import *

In [8]:
contours = contours[(contours['orientation'] == 'sagittal') & (contours['downsample'] == 1)]
contours = convert_annotation_v3_original_to_aligned_cropped(contours, stack=stack)

In [10]:
grouped = contours.groupby('section')

In [15]:
for sec, group in grouped:
    for contour_id, contour in group.iterrows():
        print contour_id
    print '\n'

24041485827291


178638821776884


225468065711699


82753828715844


108639196063443
240450955921646


139995532033938
276274803125745


123201065869455
50599366457376


128891739813762
280008487571593
74734208874016


137576545852286
139884971202912
44979814970993


198648392198851
252434421302035
53849871852705


215403441631652
272974593078035
32236200064177


165920538830976
243256546173591
58906621711441


175994136617351
258516032496448
45531577507434


187863059163195
193809479990018
245020993386827


124244720326548
28990358901640
6039163938366


13783358246531
269642137367206
93951936437221


170927976283983
250086639456848
65686469868581


12994163947428
131268714517286
180619860209674
3348124243740


140881450963675
171250512966690
277597751075951
90654685894305


124892661563969
202987135024252
207964402507211
50422758984114
63049897456070


236059505041962
259084303736308
69129898116566
86007295995378
95926547893398


111731759580942
135434889301284
265153680649971
273505